In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [ ]:
import gzip
import json
#import simplejson as json

In [ ]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [ ]:
scripts[:2]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [ ]:
practices[:10]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [ ]:
import math as mt
def describe(key):

    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    L1 = []
    N = len(scripts)
    for i in range(N):
        L = scripts[i]
        L1.append(L[key])
    total=sum(L1)
    avg = total/float(N)
    for i in range(N):
        s+=(L1[i]-avg)**2
    s = mt.sqrt(s/float(N))
    L1.sort()
    N1 = round(N/4)-1
    N2 = round(N/2)-1
    N3 = round((3*N)/4)-1
    q25 = L1[N1]
    med = L1[N2]
    q75 = L1[N3]
    
    
    return (total, avg, s, q25, med, q75)

In [ ]:
describe('items')

In [ ]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [ ]:
grader.score.pw__summary_statistics(summary)

## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [ ]:
D = set()
N = len(scripts)
for i in range(N):
    L = scripts[i]
    D.add(L['bnf_name'])
bnf_names = list(D)
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [ ]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    key = script['bnf_name']
    ite = script['items']
    groups[key].append(ite)
#print(groups)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [ ]:
max_item = []
m=0
list_tuple =[]
for key in groups:
    t= tuple()
    t = key, sum(groups[key])
    list_tuple.append(t)
for i in range(len(list_tuple)):
    x,y = list_tuple[i]
    if y>=m:
        r,m = x,y
tu=(r,m)
max_item.append(tu)
print(max_item)
print(len(list_tuple))

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [ ]:
grader.score.pw__most_common_item(max_item)

**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [ ]:
D=set()
for practice in practices:
    D.add(practice['code'])
list_practice=list(D)
list_practice.sort()
groups_code = {name: [] for name in list_practice}
for practice in practices:
    key = practice['code']
    pcode = practice['post_code']
    groups_code[key].append(pcode)
#print(groups_code)

for key in groups_code:
    if len(groups_code[key])>1:
        x = min(groups_code[key])
        groups_code[key]= list()
        groups_code[key].append(x)
for key in groups_code:
    x=groups_code[key][0]
    groups_code[key]=x
        


In [ ]:
#print(groups_code)

In [ ]:
S = set()
for key in groups_code:
    S.add(groups_code[key])
list_post_code=list(S)
list_post_code.sort()
#print(list_post_code)

In [ ]:
practice_postal = dict()
for elment in list_post_code:
    for key in groups_code:
        if groups_code[key]==elment:
            practice_postal[key]=elment

In [ ]:
#print(practice_postal)
sorted(practice_postal.values())
#practice_postal['K82019']

In [ ]:
#print(practice_postal)

In [ ]:
for practice in practice_postal:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']]=
    else:
        practice_postal[practice['code']]
practice_postal

In [ ]:
len(list_practice)

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [ ]:
assert groups_code['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [ ]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [ ]:
post_set = set()
a=0
practice_postal_100 = dict()

for key in practice_postal:
    if a<100:
        practice_postal_100[key] = practice_postal[key]
        post_set.add(practice_postal[key])
    else:
        break
    a = len(post_set)

In [ ]:
print(practice_postal)
ad=list(practice_postal)
ad.sort()
print(ad)
print(len(ad))

In [ ]:
practice_postal['M85078']

In [ ]:
L1=list(practice_postal)
print(len(L1))

In [ ]:
set_practice = set()
for script in scripts:
    set_practice.add(script['practice'])
list_practice_scripts = list(set_practice)
print(len(scripts), len(list_practice_scripts))

In [ ]:
groups_practice_scripts = {name: [] for name in list_practice_scripts}
for script in scripts:
    key = script['practice']
    ite = script['items']
    groups_practice_scripts[key].append(ite)
#print(groups_practice_scripts)

In [ ]:
tuple_practice_scripts =[]
for key in groups_practice_scripts:
    t= tuple()
    t = key, sum(groups_practice_scripts[key])
    tuple_practice_scripts.append(t)
#print((tuple_practice_scripts))
tuple_practice_scripts.sort()
#print('###############################################################')
#print(tuple_practice_scripts)

In [ ]:
final = list()
for key in practice_postal:
    for x in tuple_practice_scripts:
        code,ite = x
        if code==key:
            t=tuple()
            t=practice_postal[key],ite
            final.append(t)
print(code,ite)

In [ ]:
#print((final))

In [ ]:
set_final = set()
for el in final:
    x,y=el
    set_final.add(x)
print(len(set_final))
Lfinal = list(set_final)
Lfinal.sort()
#print(Lfinal)

In [ ]:
list_set_final = dict()
for el in Lfinal:
    list_set_final[el]=0
#print(list_set_final)
for key in list_set_final:
    for el in final:
        x,y=el
        if x==key:
            list_set_final[key]=list_set_final[key]+y
#print('################################')
#print(list_set_final)


In [ ]:
a=0
postal_total=list()
for key in list_set_final:
    t=tuple()
    t=key,list_set_final[key]
    postal_total.append(t)
postal_totals=postal_total[:100]

In [ ]:
print(postal_totals)

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [ ]:
items_by_post = ...

In [ ]:
#postal_totals = [('B11 4BW', 20673)] * 100

grader.score.pw__postal_totals(postal_totals)

## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [ ]:
print(len(bnf_names)*len(list_practice_scripts))
groups_practice_codepostal = dict()
for key in list_practice_scripts:
    for code in practice_postal:
        if key==code:
            groups_practice_codepostal[code]=practice_postal[code]
            

In [ ]:
list_100 = list()
for el in postal_totals:
    x,y=el
    list_100.append(x)
#print(list_100)

In [ ]:
#sorted(groups_practice_codepostal.keys())
#print((groups_practice_codepostal))

In [ ]:
#len(list_practice_codepostal)
#Combining bnf_names and list_practice_codepostal
list_bnf_post_code = list()

for key in groups_practice_codepostal:
    for name in bnf_names:
        t=tuple()
        t=key,groups_practice_codepostal[key],name
        list_bnf_post_code.append(t)
print(len(list_bnf_post_code))

In [ ]:
groups_practice_scripts_bnf = {name: [] for name in list_practice_scripts}
for script in scripts:
    key = script['practice']
    t=tuple()
    t=script['bnf_name'],script['items']
    groups_practice_scripts_bnf[key].append(t)
print(len(groups_practice_scripts_bnf))

In [ ]:
s=0
for key in groups_practice_scripts_bnf:
    x=len(groups_practice_scripts_bnf[key])
    s+=x
print(s)

In [ ]:
list_tuple_codepost_bnf_item = list()
for key in groups_practice_scripts_bnf:
    t=tuple
    t=practice_postal[key],groups_practice_scripts_bnf[key]
    list_tuple_codepost_bnf_item.append(t)

In [ ]:
print(len(list_tuple_codepost_bnf_item))

In [ ]:
#groups_practice_scripts_bnf['N81013']

In [ ]:
#print(list_tuple_codepost_bnf_item)

total_items_by_bnf_post = list()
for el in list_tuple_codepost_bnf_item:
    x,y = el
    new_set = set()
    for el2 in y:
        x2,y2 = el2
        new_set.add(x2)
    l_new = list(new_set)
    l_new.sort()
    for el3 in l_new:
        ite=0
        t=tuple()
        for el4 in y:
            x4,y4=el4
            if x4==el3:
                ite+=y4
        t=x,el3,ite
        total_items_by_bnf_post.append(t)
    

In [ ]:
print((total_items_by_bnf_post[:2]))

In [ ]:
l1 = list()
for el in total_items_by_bnf_post:
    x,y,z=el
    t=tuple()
    t=x,y
    l1.append(t)
print(l1[:2])

In [ ]:
len(total_items_by_bnf_post)

In [ ]:
G = zip(range(100),list_100)
G = dict(G)
print(G)

In [ ]:
list_new = list()
for el in total_items_by_bnf_post:
    x,y,z=el
    if x in G.values():
        list_new.append(el)

In [ ]:
print(len(list_new))

In [ ]:
G2 = { value : [] for value in G.values() }
G2

In [ ]:
G2 = { value : [] for value in G.values() }
for key in G2:
    for el in total_items_by_bnf_post:
        x,y,z=el
        if key==x:
            t=tuple()
            t=y,z
            G2[key].append(t)
            

In [ ]:
G2

In [ ]:
list_max = []
for key in G2:
    s=set()
    for el in G2[key]:
        y,z=el
        s.add(y)
    l=list(s)
    l.sort()
    l2=list()
    for el in l:
        t=tuple()
        ite=0
        for el2 in G2[key]:
            y,z=el2
            if y==el:
                ite+=z
        t=el,ite
        l2.append(t)
    bnf_max='a'
    m=0
    for el in l2:
        y,z=el
        if z>m:
            m=z
            bnf_max=y
    t2=key,bnf_max,m
    list_max.append(t2)

In [ ]:
total_items_by_bnf_post = ...
assert len(total_items_by_bnf_post) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [ ]:
total_items = ...
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [ ]:
max_item_by_post = list_max
max_item_by_post

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [ ]:
items_by_region = list()
for i in range(100):
    t=tuple()
    x,y,z=max_item_by_post[i]
    y2,z2=postal_totals[i]
    z3=z/float(z2)
    t=x,y,z3
    items_by_region.append(t)

In [ ]:
grader.score.pw__items_by_region(items_by_region)

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*